In [3]:
import pandas as pd
import numpy as np
import requests
import ast
import mariadb
import sys
import re
from logger.Logger import etlLogger
from datetime import datetime, timedelta
from modulos.auxiliares import *
from modulos.conecta_db import *
from modulos.crawler import *
from modulos.data_processing import *



In [6]:
def connect_db(LOGGER_OBJ):
    LOGGER_OBJ.info('Conectando ao banco de dados')
    cred = read_yaml('config/config.yaml')
    try:
        conn = mariadb.connect(
            user=cred['DB']['USER'],
            password=cred['DB']['PSSWRD'],
            host="localhost",
            port=3306,
            database="ALBION"
        )
        LOGGER_OBJ.info("Conexão estabelecida")
    except mariadb.Error as e:
        LOGGER_OBJ.error(f"Falha na conexão: {e}")
        sys.exit(1)

    cur = conn.cursor()

    return conn, cur

In [7]:
LOGGER_OBJ = etlLogger(project_name='test')

conn, cur = connect_db(LOGGER_OBJ)

{"timestamp": "2022-08-25 10:33:34,052", "processo": "MainProcess", "thread": "MainThread","level": "INFO", "script": "2023394076.py", "modulo": "2023394076", "metodo": "connect_db",  "mensagem": "Conectando ao banco de dados"}
{"timestamp": "2022-08-25 10:33:34,066", "processo": "MainProcess", "thread": "MainThread","level": "INFO", "script": "2023394076.py", "modulo": "2023394076", "metodo": "connect_db",  "mensagem": "Conexão estabelecida"}


In [8]:
def return_datetime():
    dt_today = datetime.now()
    dt_tomorrow = datetime.now() + timedelta(days=1)
    str_date_today = dt_today.strftime("%m-%d-%Y")
    str_date_tomorrow = dt_tomorrow.strftime("%m-%d-%Y")

    return str_date_today, str_date_tomorrow

In [9]:
def get_urls():
    cities, itens = cities_and_itens()
    dt_today, dt_tomorrow = return_datetime()
    dict_anchor = {'today': dt_today, 'tomorrow': dt_tomorrow, 'itens': itens, 'cities': cities}
    cred = read_yaml("config/config.yaml")
    list_urls = [cred['URLS']['URL_GOLD'], cred['URLS']['URL_ITEM']]
    new_urls = []
    for url in list_urls:
        for word, replacement in dict_anchor.items():
            url = re.sub(word, replacement, url)
        new_urls.append(url)
    
    return new_urls

In [10]:
new_urls = get_urls()

In [11]:
new_urls

['https://www.albion-online-data.com/api/v2/stats/gold?date=08-25-2022&end_date=08-26-2022',
 'https://www.albion-online-data.com/api/v2/stats/history/T4_POTION_HEAL,T6_POTION_HEAL,T7_POTION_STONESKIN,T4_POTION_COOLDOWN,T6_POTION_COOLDOWN,T4_POTION_HEAL@1,T6_POTION_HEAL@1,T7_POTION_STONESKIN@1,T4_POTION_COOLDOWN@1,T6_POTION_COOLDOWN@1?date=08-25-2022&end_date=08-26-2022&locations=Bridgewatch,Caerleon,Lymhurst,Martlock,Thetford&qualities=1,2,3&time-scale=6']